In [23]:
import os
import json
from pathlib import Path
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt


from data import get_dataset, get_zoo_elephants_images_and_labels, get_ELEP_images_and_labels
# from train import SiameseModel
from metrics import get_kernel_mask, val
from model import get_model
from triplet_loss import batch_all_triplet_loss

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras.models import load_model

In [2]:
with open('hyperparameters/initial_run.json', 'rb') as f:
    params = json.load(f)

In [3]:
params['batch_size']['train'] = 5
params['batch_size']['val'] = 5

In [4]:
data_dir = '/Users/deepakduggirala/Documents/ELPephant-cropped'

In [5]:
cache_files = {
        'train': 'train.cache',
        'val': 'val.cache'
    }

In [6]:
train_ds, N_train = get_dataset(get_ELEP_images_and_labels, params, data_dir, 'train', cache_files)
val_ds, N_val = get_dataset(get_ELEP_images_and_labels, params, data_dir, 'val', cache_files)

reading Tensor("args_0:0", shape=(), dtype=string)
reading Tensor("args_0:0", shape=(), dtype=string)


In [7]:
def val_metric(y_true, y_pred):
    labels = y_true
    embeddings = y_pred
    
    # print(labels.shape, embeddings.shape, type(labels), type(embeddings))
    return tf.reduce_sum(embeddings)

In [33]:
class SiameseModel(Model):
    def __init__(self, params, finetune):
        super().__init__()
        self.params = params
        self.finetune = finetune
        self.siamese_network = get_model(params, finetune)
        self.custom_loss = batch_all_triplet_loss
        self.loss_tracker = metrics.Mean(name="loss")
        self.val_rate_tracker = metrics.Mean(name="VAL")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        images, labels = data
        with tf.GradientTape() as tape:
            loss, embeddings = self._compute_loss(images, labels) 
            loss = loss + sum(self.siamese_network.losses)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result(), "VAL": self.val_rate_tracker.result()}

    def test_step(self, data):
        images, labels = data
        loss, embeddings = self._compute_loss(images, labels)

        K, mask = get_kernel_mask(labels.numpy(), embeddings.numpy())
        d = self.params['margin'] if self.params['squared'] else self.params['margin']**2
        val_rate = val(K, mask, d, include_diag=True)
        self.val_rate_tracker.update_state(val_rate)
        
        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result(), "VAL": self.val_rate_tracker.result()}

    def _compute_loss(self, images, labels):
        embeddings = self.siamese_network(images)
        embeddings = tf.math.l2_normalize(embeddings, axis=1, epsilon=1e-10)
        
        return self.custom_loss(labels, embeddings, self.params['margin'], self.params['squared']), embeddings

In [30]:
siamese_model = SiameseModel(params, False)
siamese_model.compile(optimizer=optimizers.Adam(params['lr']), run_eagerly=True)

In [31]:
siamese_model.fit(train_ds.take(1),
                      epochs=2,
                      validation_data=val_ds)

Epoch 1/2
1/1 [==============================] - 27s 27s/step - loss: 4.5506 - VAL: 0.0000e+00 - val_loss: 0.0378 - val_VAL: 0.9551
Epoch 2/2
1/1 [==============================] - 26s 26s/step - loss: 4.5241 - VAL: 0.0000e+00 - val_loss: 0.0468 - val_VAL: 0.9510
